#Data

In [339]:
import numpy as np
import pandas as pd

In [340]:
news = pd.read_csv('News_Final.csv')
news.head()

,IDLink,Title,Headline,Source,Topic,PublishDate,SentimentTitle,SentimentHeadline,Facebook,GooglePlus,LinkedIn
0,99248.0,Obama Lays Wreath at Arlington National Cemetery,Obama Lays Wreath at Arlington National Cemete...,USA TODAY,obama,2002-04-02 00:00:00,0.000000,-0.053300,-1,-1,-1
1,10423.0,A Look at the Health of the Chinese Economy,"Tim Haywood, investment director business-unit...",Bloomberg,economy,2008-09-20 00:00:00,0.208333,-0.156386,-1,-1,-1
2,18828.0,Nouriel Roubini: Global Economy Not Back to 2008,"Nouriel Roubini, NYU professor and chairman at...",Bloomberg,economy,2012-01-28 00:00:00,-0.425210,0.139754,-1,-1,-1
3,27788.0,Finland GDP Expands In Q4,Finland's economy expanded marginally in the t...,RTT News,economy,2015-03-01 00:06:00,0.000000,0.026064,-1,-1,-1
4,27789.0,"Tourism, govt spending buoys Thai economy in J...",Tourism and public spending continued to boost...,The Nation - Thailand&#39;s English news,economy,2015-03-01 00:11:00,0.000000,0.141084,-1,-1,-1


In [341]:
news = news.drop(columns=["IDLink", "Headline", "Source", "Topic", "PublishDate", "SentimentHeadline", "Facebook", "GooglePlus", "LinkedIn"])
news.head()

,Title,SentimentTitle
0,Obama Lays Wreath at Arlington National Cemetery,0.000000
1,A Look at the Health of the Chinese Economy,0.208333
2,Nouriel Roubini: Global Economy Not Back to 2008,-0.425210
3,Finland GDP Expands In Q4,0.000000
4,"Tourism, govt spending buoys Thai economy in J...",0.000000


In [342]:
# bestNews = news[news["SentimentTitle"] == 1]
# for i in np.array(bestNews['Title']):
#   print(i)
# news['SentimentHeadline'][2]

In [343]:
for i in range(len(news['SentimentTitle'])):
  if news['SentimentTitle'][i] >= 0.0:
    news['SentimentTitle'][i] = 1.0
  # elif news['SentimentTitle'][i] < 0.25 and news['SentimentTitle'][i] > -0.25:
  #   news['SentimentTitle'][i] = 0.0
  else:
    news['SentimentTitle'][i] = 0.0
news['SentimentTitle'] = news['SentimentTitle'].astype(int, errors='ignore')

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys


In [344]:
news.tail()

,Title,SentimentTitle
93234,Stocks rise as investors key in on US economy ...,1
93235,Russian PM proposes to use conservative and to...,1
93236,Palestinian Government Uses Foreign Aid To Pay...,1
93237,Palestine Youth Orchestra prepares for first U...,1
93238,Sausalito businesswoman wins $10000 in Microso...,1


In [345]:
sentimentArr = np.array(pd.Series(news['SentimentTitle']))
headlineArr = np.array(pd.Series(news['Title']))
sentimentArr

array([1, 1, 0, ..., 1, 1, 1])

In [346]:
print("Max: ", np.amax(sentimentArr), ", Min: ", np.amin(sentimentArr))

Max:  1 , Min:  0


In [347]:
from sklearn.model_selection import train_test_split
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

In [348]:
X_train, X_test, y_train, y_test = train_test_split(news['Title'], news['SentimentTitle'], test_size=0.33)

#Model Training

In [349]:
!nvidia-smi

Thu Jul 29 03:51:42 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.42.01    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   50C    P0    27W /  70W |    382MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [350]:
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
tokenizer = Tokenizer(oov_token="<OOV>")

In [351]:
vocab_size = 40000
embedding_dim = 16
max_length = 120
trunc_type = 'post'
oov_tok = '<OOV>'
padding_type = 'post'

In [352]:
training_sentences = []
testing_sentences = []

for i in X_train:
    training_sentences.append(str(i))
for i in X_test:
    testing_sentences.append(str(i))

In [353]:
tokenizer = Tokenizer(num_words=vocab_size, oov_token=oov_tok)
tokenizer.fit_on_texts(training_sentences)
word_index = tokenizer.word_index

In [354]:
sequences = tokenizer.texts_to_sequences(training_sentences)
padded = pad_sequences(sequences, maxlen=max_length, truncating=trunc_type)
testing_sentences = tokenizer.texts_to_sequences(testing_sentences)
testing_padded = pad_sequences(testing_sentences, maxlen=max_length)

In [355]:
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim,       input_length=max_length),
    tf.keras.layers.GlobalAveragePooling1D(),
    tf.keras.layers.Dense(6, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

In [356]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [357]:
model.summary()

Model: "sequential_12"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_12 (Embedding)     (None, 120, 16)           640000    
_________________________________________________________________
global_average_pooling1d_12  (None, 16)                0         
_________________________________________________________________
dense_24 (Dense)             (None, 6)                 102       
_________________________________________________________________
dense_25 (Dense)             (None, 1)                 7         
Total params: 640,109
Trainable params: 640,109
Non-trainable params: 0
_________________________________________________________________


In [358]:
training_labels_final = np.array(y_train)
testing_labels_final = np.array(y_test)

In [359]:
model.fit(padded, training_labels_final, epochs=7, validation_data=(testing_padded, testing_labels_final))

Epoch 1/7
1953/1953 [==============================] - 18s 9ms/step - loss: 0.6621 - accuracy: 0.6014 - val_loss: 0.6025 - val_accuracy: 0.7741
Epoch 2/7
1953/1953 [==============================] - 17s 9ms/step - loss: 0.4737 - accuracy: 0.7947 - val_loss: 0.4122 - val_accuracy: 0.8212
Epoch 3/7
1953/1953 [==============================] - 18s 9ms/step - loss: 0.3525 - accuracy: 0.8616 - val_loss: 0.3613 - val_accuracy: 0.8536
Epoch 4/7
1953/1953 [==============================] - 16s 8ms/step - loss: 0.2972 - accuracy: 0.8903 - val_loss: 0.3596 - val_accuracy: 0.8613
Epoch 5/7
1953/1953 [==============================] - 17s 9ms/step - loss: 0.2605 - accuracy: 0.9082 - val_loss: 0.3285 - val_accuracy: 0.8701
Epoch 6/7
1953/1953 [==============================] - 18s 9ms/step - loss: 0.2344 - accuracy: 0.9200 - val_loss: 0.3497 - val_accuracy: 0.8585
Epoch 7/7
1953/1953 [==============================] - 18s 9ms/step - loss: 0.2144 - accuracy: 0.9286 - val_loss: 0.3140 - val_accuracy:

#Testing

In [365]:
def modelFunc(prompt: str):
  arr = [prompt]
  arrtokens = tokenizer.texts_to_sequences(arr)
  newarr = pad_sequences(arrtokens, maxlen=max_length, truncating=trunc_type)
  result = model.predict(newarr)
  if result >= 0.5:
    return 1
  else:
    return 0

In [361]:
import time
def populateList(link: str, requests: int, after: str, count: int):
  print(requests)
  if requests > 0:
    headlines = []
    if after == "" and count == 0:
      df = pd.read_json(link)
    else:
      df = pd.read_json(link + "&after=" + after + "&count=" + str(count))
      print(link + "?after=" + after + "?count=" + str(count))
    for i in range (0, df['data']['dist']):
      headlines.append(df['data']['children'][i]['data']['title']) 
    time.sleep(10)
    for item in populateList(link, requests - 1, df['data']['after'], int(df['data']['dist'])+count):
      headlines.append(item)
    res = []
    for i in headlines:
      if i not in res:
        res.append(i)
    return res
  else:
    return []
headlinesPositive = populateList("https://www.reddit.com/r/UpliftingNews/hot/.json?limit=100", 1, "", 0)

1
0


In [366]:
veryPositive = []
for i in headlinesPositive:
  result = modelFunc(prompt=i)
  if result >= 0.5:
    result = 1.0
    print(i)
    veryPositive.append(i)
  else:
    result = 0.0
  int(result)

"This isn't uplifting!"
Last privately-owned, undeveloped barrier island on NC coast is now protected forever
‘South Park’ Creators Aim to Buy Bankrupt Casa Bonita, Restaurant Featured in Show
BioNTech to develop mRNA malaria vaccine; potentially saving 100's of thousands of lives per year
Bob Odenkirk In Stable Condition After “Heart Related Incident” On ‘Better Call Saul’ Set, Says Rep
Oregon governor signs ambitious clean energy bill. According to the governor's office it sets an "aggressive timeline" for moving to 100% clean electricity sources by 2040
The ‘world’s most powerful tidal turbine’ starts to export power to the grid
Renewables now half the price of fossil fuels across Europe, says report
Blink-182’s Mark Hoppus Felt Well Enough To Play Bass For The First Time Since His Cancer Diagnosis
Walmart to pay for 100% of college tuition, books for employees
A birthday gift: Israeli woman donates kidney to Gaza boy
Israel, UAE to swap kidneys in historic life-saving operation
Ins

In [367]:
len(veryPositive)

61

In [375]:
modelFunc("PROMPT")

1